In [2]:
#need these for plotting in notebook or file
from bokeh.io import output_file, output_notebook, reset_output, show
#this stops outputting to a file just in case it has been started
reset_output()
#this puts output within notebook
output_notebook()

Loading BokehJS ...

In [3]:
#creating interactive stuff in bokeh

from bokeh.layouts import widgetbox
from bokeh.models.widgets import Button, RadioButtonGroup, Select, Slider

#if want to plot to a file use this
#output_file("layout_widgets.html")

# create some widgets
slider = Slider(start=0, end=10, value=1, step=.1, title="Slider")
button_group = RadioButtonGroup(labels=["Option 1", "Option 2", "Option 3"], active=0)
select = Select(title="Option:", value="foo", options=["foo", "bar", "baz", "quux"])
button_1 = Button(label="Button 1")
button_2 = Button(label="Button 2")

# put the results in a row
show(widgetbox(button_1, slider, button_group, select, button_2, width=300))

In [52]:
#trying to get interactive stuff to return variables

from bokeh.models import Select
#what is value here?
census_menu = Select(options=['census_var1', 'census_var2', 'census_var3'],
                value='c', title='Census Variables')
ycom_menu = Select(options=['ycom_var1', 'ycom_var2', 'ycom_var3'],
                value='v', title='YCOM Variables') 
controls = column(census_menu, ycom_menu)
show(controls)

# Add callback to widgets
# Don't have this working yet
def callback(attr, old, new):
    if census_menu.value == 'census_var1': f = 1
    elif census_menu.value == 'census_var2': f = 2
    else: f = 3
    print(f)
    #source.data={'x': f(size=N), 'y': f(size=N)}
    census_menu.on_change('value', callback) 
    print(f)
    

In [7]:
#practicing layouts in bokeh
#https://bokeh.pydata.org/en/latest/docs/user_guide/layout.html
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.palettes import Viridis3

x = list(range(11))
y0 = x
y1 = [10 - i for i in x]
y2 = [abs(i - 5) for i in x]

# create three plots
p1 = figure(plot_width=250, plot_height=250, title=None)
p1.circle(x, y0, size=10, color=Viridis3[0])
p2 = figure(plot_width=250, plot_height=250, title=None)
p2.triangle(x, y1, size=10, color=Viridis3[1])
p3 = figure(plot_width=250, plot_height=250, title=None)
p3.square(x, y2, size=10, color=Viridis3[2])

# make a grid
grid = gridplot([[p1, None], [widgetbox(button_1, slider, button_group, select, button_2, width=300), p3]])

# show the results
show(grid)

In [9]:
# another example of getting layout sorted in bokeh

import numpy as np
from bokeh.plotting import figure
from bokeh.models import Spacer, Paragraph
from bokeh.layouts import row, column, widgetbox
from bokeh.io import show

N = 10
x = np.linspace(0, 4 * np.pi, N)
y = np.sin(x)
options = dict(tools="", toolbar_location=None, plot_height=300, plot_width=300)
p2 = figure(title="Annular wedge (100 x 300)", title_location='right', **options)
p2.plot_width = 100
p2.annular_wedge(x, y, 10, 20, 0.6, 4.1, inner_radius_units="screen", outer_radius_units="screen")

p3 = figure(title="Bezier (300 x 300)", **options)
p3.bezier(x, y, x + 0.4, y, x + 0.1, y + 0.2, x - 0.1, y - 0.2)

p4 = figure(title="Quad (300 x 300)", **options)
p4.quad(x, x - 0.2, y, y - 0.2)

spacer_1 = Spacer(width=100, height=100)
spacer_2 = Spacer(width=300, height=100)
paragraph = Paragraph(text="We build up a grid plot manually. Try changing the mode yourself.")

MODE = 'fixed'
widgets = widgetbox([paragraph], sizing_mode=MODE)
row_1 = row([spacer_1, p1, spacer_2], sizing_mode=MODE)
row_2 = row([p2, p3, p4], sizing_mode=MODE)
layout = column([widgets, row_1, row_2], sizing_mode=MODE)

show(layout)

In [50]:
# Example of working updating graph
# Going to try to get this working using dropdowns in next cell
from bokeh.layouts import column
from bokeh.models import CustomJS, ColumnDataSource, Slider

x = [x*0.005 for x in range(0, 201)]

source = ColumnDataSource(data=dict(x=x, y=x))

plot = figure(plot_width=400, plot_height=400)
plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6)

slider = Slider(start=0.1, end=6, value=1, step=.1, title="power")

update_curve = CustomJS(args=dict(source=source, slider=slider), code="""
    var data = source.data;
    var f = slider.value;
    x = data['x']
    y = data['y']
    for (i = 0; i < x.length; i++) {
        y[i] = Math.pow(x[i], f)
    }
    
    // necessary becasue we mutated source.data in-place
    source.change.emit();
""")
slider.js_on_change('value', update_curve)


show(column(slider, plot))

In [58]:
# Ok, hopefully we can get this working with dropdowns
# Sweet. Works.
from bokeh.layouts import column
from bokeh.models import CustomJS, ColumnDataSource, Slider
from bokeh.models import Select

x = [x*0.005 for x in range(0, 201)]

source = ColumnDataSource(data=dict(x=x, y=x))
#source = ColumnDataSource(df) #there is an option for putting in a dataframe as a source
#see https://bokeh.pydata.org/en/latest/docs/reference/models/sources.html

plot = figure(plot_width=400, plot_height=400)
plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6)

#dropdown
ycom_menu = Select(options=['1', '2', '5','0.1'],
                value='v', title='YCOM Variables') 

update_curve = CustomJS(args=dict(source=source, ycom_menu=ycom_menu), code="""
    var data = source.data;
    var f = ycom_menu.value;
    x = data['x']
    y = data['y']
    for (i = 0; i < x.length; i++) {
        y[i] = Math.pow(x[i], f)
    }
    
    // necessary becasue we mutated source.data in-place
    source.change.emit();
""")
ycom_menu.js_on_change('value', update_curve)


show(column(ycom_menu, plot))